In [82]:
import pandas as pd
import numpy as np
import os

from folium.features import DivIcon
import folium
import pyproj
from pyproj import Proj, transform

%matplotlib inline  
import matplotlib.pyplot as plt 
import seaborn as sns 

import warnings
warnings.filterwarnings(action='ignore')

def katec_to_wgs84(x, y):
    inProj  = Proj('+proj=tmerc +lat_0=38 +lon_0=128 +k=0.9999 +x_0=400000 +y_0=600000 +ellps=bessel +units=m +no_defs +towgs84=-115.80,474.99,674.11,1.16,-2.31,-1.63,6.43')
    outProj = Proj({ 'proj':'latlong', 'datum':'WGS84', 'ellps':'WGS84' })
    return transform( inProj, outProj, x, y )

WGS84 = { 'proj':'latlong', 'datum':'WGS84', 'ellps':'WGS84', }
KATEC = { 'proj':'tmerc', 'lat_0':'38N', 'lon_0':'128E', 'ellps':'bessel','x_0':'400000', 'y_0':'600000', 'k':'0.9999','towgs84':'-115.80,474.99,674.11,1.16,-2.31,-1.63,6.43'}
def wgs84_to_katec(longitude, latitude):
    return transform( Proj(**WGS84), Proj(**KATEC), longitude, latitude )

##### 현재 작업 위치 확인


In [83]:
path = "C:/Users/dudtj/busan_moving_person/data/opt_data"

In [86]:
os.chdir(path)
file_list = os.listdir(path)
f_list = [file for file in file_list if file.endswith(".csv")]
f_list

['CV100_pr_opt_analysis.csv', 'pr_opt_analysis.csv']

In [88]:
optDF = pd.read_csv(f_list[0], encoding="UTF-8")
optDF

,id,x,y,m,f,total,m_index,f_index
0,49242847,492425,284725,33482.95,43484.59,76967.54,5.71,7.42
1,49252827,492525,282725,792.63,886.29,1678.92,6.20,6.93
2,49252847,492525,284725,133931.80,173938.36,307870.16,5.71,7.42
3,49252848,492525,284825,30497.70,26058.41,56556.11,7.08,6.05
4,49262828,492625,282825,33062.38,34197.22,67259.60,6.45,6.67
...,...,...,...,...,...,...,...,...
2414,49872862,498725,286225,42739.22,69309.69,112048.91,5.01,8.12
2415,49872863,498725,286325,30005.10,31090.55,61095.65,6.45,6.68
2416,49872864,498725,286425,28007.71,23585.56,51593.27,7.12,6.00
2417,49882848,498825,284825,24375.65,23619.81,47995.46,6.67,6.46


In [131]:
tdf = pd.DataFrame(optDF['f_index'].groupby(optDF['id']).sum()).reset_index()
geodf = pd.merge(tdf, optDF[['id', 'x', 'y']], how = 'left', on = 'id')[['id', 'x', 'y', 'f_index']].drop_duplicates()

geodf.to_csv('./100c_opt_analysis_dense_f.csv', index=False, encoding='UTF-8')
# geodf['round_m_index'].sort_values(ascending=False)[:10].values

In [132]:
geodf.describe()

,id,x,y,f_index
count,2.419000e+03,2419.000000,2419.000000,2419.000000
mean,4.955714e+07,495567.910294,285295.814386,6.745635
std,1.567794e+05,1567.766967,1423.636505,0.695150
min,4.924285e+07,492425.000000,282425.000000,3.100000
25%,4.943283e+07,494325.000000,284225.000000,6.500000
50%,4.955286e+07,495525.000000,285225.000000,6.740000
75%,4.968283e+07,496825.000000,286325.000000,6.990000
max,4.988285e+07,498825.000000,289425.000000,9.960000


In [134]:
geodf['f_index'].sort_values(ascending=False).values[:50]

array([9.96, 9.94, 9.89, 9.87, 9.87, 9.84, 9.82, 9.81, 9.8 , 9.78, 9.75,
       9.74, 9.69, 9.69, 9.67, 9.65, 9.64, 9.61, 9.6 , 9.6 , 9.54, 9.47,
       9.4 , 9.26, 9.19, 9.06, 9.02, 9.01, 8.98, 8.96, 8.95, 8.94, 8.91,
       8.86, 8.63, 8.62, 8.62, 8.6 , 8.59, 8.59, 8.53, 8.53, 8.52, 8.52,
       8.51, 8.51, 8.51, 8.51, 8.5 , 8.5 ])

In [160]:
def f_color(x):
    if x in geodf['f_index'].sort_values(ascending=False).values[230:250]: # 60
        a = '#FF4F00'
    elif x in geodf['f_index'].sort_values(ascending=False)[250:290].values:
        a = '#FCB100'   
    elif x in geodf['f_index'].sort_values(ascending=False)[290:340].values:      # 50%를 기준으로 설정
        a = '#E0F500' 
    elif x in geodf['f_index'].sort_values(ascending=False)[340:470].values:      # 25%를 기준으로 설정
        a = '#8CF700'  
    else:
        a = '#808080'
    return a

In [161]:
geodf['cell_color']= geodf['f_index'].apply(lambda x : f_color(x))

print("function success :)")

###################################################################
### 셀 수 변경###
cell_size = 100 #
#################
a = cell_size//2
x = geodf
x['nwx'], x['nwy'] = x['x']-a, x['y']+a
x['nex'], x['ney'] = x['x']+a, x['y']+a
x['swx'], x['swy'] = x['x']-a, x['y']-a
x['sex'], x['sey'] = x['x']+a ,x['y']-a

x['lng'], x['lat'] = katec_to_wgs84(x.x.to_list(), x.y.to_list())
x['nwlng'], x['nwlat'] = katec_to_wgs84(x.nwx.to_list(), x.nwy.to_list())
x['nelng'], x['nelat'] = katec_to_wgs84(x.nex.to_list(), x.ney.to_list())
x['swlng'], x['swlat'] = katec_to_wgs84(x.swx.to_list(), x.swy.to_list())
x['selng'], x['selat'] = katec_to_wgs84(x.sex.to_list(), x.sey.to_list())

map = folium.Map(location=[geodf['lat'].mean(), geodf['lng'].mean()], zoom_start=13, tiles='stamentoner')

for n in x.index:
    nw, ne, sw, se = [x.loc[n, 'nwlat'],x.loc[n, 'nwlng']],[x.loc[n, 'nelat'],x.loc[n, 'nelng']],[x.loc[n, 'swlat'],x.loc[n, 'swlng']],[x.loc[n, 'selat'],x.loc[n, 'selng']]
    cell_color = x.loc[n, 'cell_color']
    loc_name = 'ID:'+str(round(x.loc[n, 'id'],3)) + '       f_index:'+ str(round(x.loc[n, 'f_index'],3))
    folium.Polygon(locations=[nw, sw, se, ne],
                   color='white',
                   popup=loc_name,
                   weight=0.55,
                   fill=True,
                   fill_color=cell_color,
                   fill_opacity=0.6
     ).add_to(map)

map.save('./test_top200_opt_analysis_dense_f.html') ###### 이름변경필요부분
# map
print("done :)")


function success :)
done :)


In [56]:
print(geodf['total'].sort_values(ascending=False)[:50].values.max())# 60
print(geodf['total'].sort_values(ascending=False)[50:100].values.max()) # 36
print(geodf['total'].sort_values(ascending=False)[100:150].values.max()) # 28
print(geodf['total'].sort_values(ascending=False)[150:200].values.max()) # 24

984
48
35
27
